# Calculating the LSD profiles 

## 0 Packages and data

In [1]:
path='G:/Shared drives/BeStarsMiMeS/ScienceFiles/' #Patrick's google file stream path

In [2]:

import LSDpy.lsdpy as lsdpy


In [3]:
import specpolFlow as pol
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const

In [4]:
from matplotlib.backends.backend_pdf import PdfPages

import pandas as pd
import itertools

### This gets rid of the warnings in the cells where it is creating more than 20 figures (i.e. 1.2)
#### Not needed for this notebook but could be useful in the future
#import warnings
#warnings.filterwarnings("default", message="More than 20 figures have been opened.")

In [5]:
import pandas as pd
sheet_id = '1M6y1Wnsrc-w5FjUMfKaSFa_-foIDAaMe8W4lYNWnWyk'
sheet_name = 'Stars'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
StarData=pd.read_csv(url)

sheet_name = 'Observations'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
Observations=pd.read_csv(url)

### 2.1 **Spectra**: Synth-hybrid-spectra
reran - Patrick


In [6]:
df = pd.read_csv('{}03-LSD-calculations/LSD/Params-LSDProfiles-CalculatedByPython.csv'.format(path))


#### **Mask**: tellClean Mask depth 0.02:

#### **Mask**: tellClean Mask depth 0.01:

In [7]:
%%capture
with PdfPages('{}03-LSD-calculations/hybrid_maskdepth0.01_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    file = '{}01-Synth-calculations/Synth-hybrid-spectra/{}_{}_noise.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/hybrid_maskdepth0.01_tellClean/{}_{}.lsd'.format(path,star,obs)
    mask='{}02-Mask-calculations/MaskCleaning/depth0.01_tellClean/{}_{}_depth0.01.mask'.format(path,modelcode,star)

    lsd, mod = pol.run_lsdpy(obs = file, mask = mask, outLSDName = outfile, 
            velStart = velS, velEnd = velE, velPixel = velP, 
            normDepth = 0.2, normLande = 1.2, normWave = 500.0)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

In [40]:
%%capture
with PdfPages('{}03-LSD-calculations/hybrid_maskdepth0.02_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    file = '{}01-Synth-calculations/Synth-hybrid-spectra/{}_{}_noise.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/hybrid_maskdepth0.02_tellClean/{}_{}.lsd'.format(path,star,obs)
    mask='{}02-Mask-calculations/MaskCleaning/depth0.02_tellClean/{}_{}_depth0.02.mask'.format(path,modelcode,star)

    lsd, mod = pol.run_lsdpy(obs = file, mask = mask, outName = outfile, 
            velStart = velS, velEnd = velE, velPixel = velP, 
            normDepth = 0.2, normLande = 1.2, normWave = 500.0)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean Mask depth 0.1: OLD

In [9]:
%%capture
with PdfPages('{}03-LSD-calculations/hybrid__maskdepth0.1_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    for j in range(0,len(blend)):
      if blend[j]==star:
        mask = '{}02-Mask-calculations/MaskCleaning/{}_depth0.1.mask'.format(path,star)
        break
      if blend[j]!=star:
        mask = '{}02-Mask-calculations/MaskCleaning/depth0.1_tellClean/{}_depth0.1.mask'.format(path,modelcode)

    file = '{}01-Synth-calculations/Synth-hybrid-spectra/{}_{}_noise.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/hybrid_maskdepth0.1_tellClean/{}_{}.lsd'.format(path,star,obs)

    toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
            velStart=velS, velEnd=velE, velPixel=velP, 
            normDepth=0.2, normLande=1.2, normWave=500.0,
            removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
            interpMode=1, fSaveModelS=0, outModelName='',
            fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')
    lsd = pol.read_lsd(outfile)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

FileNotFoundError: G:/Shared drives/BeStarsMiMeS/UpdatedFiles/02-Mask-calculations/MaskCleaning/hd6226_depth0.1.mask not found.

#### **Mask**: tellClean CleanAsifTweak

In [41]:
%%capture
with PdfPages('{}03-LSD-calculations/hybrid_CleanAsifTweak_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    mask = '{}02-Mask-calculations/MaskCleaning/AsifMaskCleanTweak_tellClean/{}_clean_{}.mask_auto_twk'.format(path,star,obs)
    file = '{}01-Synth-calculations/Synth-hybrid-spectra/{}_{}_noise.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/hybrid_CleanAsifTweak+tellClean/{}_{}.lsd'.format(path,star,obs)

    lsd, mod = pol.run_lsdpy(obs = file, mask = mask, outName = outfile, 
            velStart = velS, velEnd = velE, velPixel = velP, 
            normDepth = 0.2, normLande = 1.2, normWave = 500.0)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean AsifClean

In [42]:
%%capture
with PdfPages('{}03-LSD-calculations/hybrid_CleanAsif_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    mask = '{}02-Mask-calculations/MaskCleaning/AsifMaskClean_tellClean/{}_clean_{}.mask'.format(path,star,obs)

    file = '{}01-Synth-calculations/Synth-hybrid-spectra/{}_{}_noise.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/hybrid_CleanAsif+tellClean/{}_{}.lsd'.format(path,star,obs)
    
    lsd, mod = pol.run_lsdpy(obs = file, mask = mask, outName = outfile, 
            velStart = velS, velEnd = velE, velPixel = velP, 
            normDepth = 0.2, normLande = 1.2, normWave = 500.0)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean SuperCleanAsifTweak

In [43]:
%%capture
with PdfPages('{}03-LSD-calculations/hybrid_SuperCleanAsifTweak_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    mask = '{}02-Mask-calculations/MaskCleaning/AsifMaskSuperCleanTweak_tellClean/{}_clean_{}.mask_auto_twk'.format(path,star,obs)

    file = '{}01-Synth-calculations/Synth-hybrid-spectra/{}_{}_noise.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/hybrid_SuperCleanAsifTweak+tellClean/{}_{}.lsd'.format(path,star,obs)

    lsd, mod = pol.run_lsdpy(obs = file, mask = mask, outName = outfile, 
            velStart = velS, velEnd = velE, velPixel = velP, 
            normDepth = 0.2, normLande = 1.2, normWave = 500.0)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean AsifSuperClean

In [8]:
%%capture
with PdfPages('{}03-LSD-calculations/hybrid_SuperCleanAsif_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    mask = '{}02-Mask-calculations/MaskCleaning/AsifMaskSuperClean_tellClean/{}_clean_{}.mask'.format(path,star,obs)

    file = '{}01-Synth-calculations/Synth-hybrid-spectra/{}_{}_noise.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/hybrid_SuperCleanAsif+tellClean/{}_{}.lsd'.format(path,star,obs)

    lsd, mod = pol.run_lsdpy(obs = file, mask = mask, outName = outfile, 
            velStart = velS, velEnd = velE, velPixel = velP, 
            normDepth = 0.2, normLande = 1.2, normWave = 500.0)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

### 2.2 **Spectra**: Normalized-Spectra
reran - Patrick


In [9]:
df = pd.read_csv('{}03-LSD-calculations/LSD/Params-LSDProfiles-CalculatedByPython.csv'.format(path))


#### **Mask**: tellClean Mask depth 0.02:

In [46]:
%%capture

with PdfPages('{}03-LSD-calculations/normalized_maskdepth0.02_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    mask = '{}02-Mask-calculations/MaskCleaning/depth0.02_tellClean/{}_{}_depth0.02.mask'.format(path,modelcode,star)

    file = '{}00-InputMaterial/NormalizedSpectra/{}_{}.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/normalized_maskdepth0.02_tellClean/{}_{}.lsd'.format(path,star,obs)

    lsd, mod = pol.run_lsdpy(obs = file, mask = mask, outName = outfile, 
            velStart = velS, velEnd = velE, velPixel = velP, 
            normDepth = 0.2, normLande = 1.2, normWave = 500.0)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean Mask depth 0.1: OLD

In [ ]:
%%capture

with PdfPages('{}03-LSD-calculations/normalized_maskdepth0.1_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    for j in range(0,len(blend)):
      if blend[j]==star:
        mask = '{}02-Mask-calculations/MaskCleaning/{}_depth0.1.mask'.format(path,star)
        break
      if blend[j]!=star:
        mask = '{}02-Mask-calculations/MaskCleaning/depth0.1_tellClean/{}_depth0.1.mask'.format(path,modelcode)

    file = '{}00-InputMaterial/NormalizedSpectra/{}_{}.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/normalized_maskdepth0.1_tellClean/{}_{}.lsd'.format(path,star,obs)

    toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
            velStart=velS, velEnd=velE, velPixel=velP, 
            normDepth=0.2, normLande=1.2, normWave=500.0,
            removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
            interpMode=1, fSaveModelS=0, outModelName='',
            fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')
    lsd = pol.read_lsd(outfile)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean CleanAsifTweak

In [47]:
%%capture
with PdfPages('{}03-LSD-calculations/normalized_CleanAsifTweak_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    mask = '{}02-Mask-calculations/MaskCleaning/AsifMaskCleanTweak_tellClean/{}_clean_{}.mask_auto_twk'.format(path,star,obs)

    file = '{}00-InputMaterial/NormalizedSpectra/{}_{}.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/normalized_CleanAsifTweak+tellClean/{}_{}.lsd'.format(path,star,obs)
    
    lsd, mod = pol.run_lsdpy(obs = file, mask = mask, outName = outfile, 
            velStart = velS, velEnd = velE, velPixel = velP, 
            normDepth = 0.2, normLande = 1.2, normWave = 500.0)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean AsifClean

In [48]:
%%capture
with PdfPages('{}03-LSD-calculations/normalized_CleanAsif_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    mask = '{}02-Mask-calculations/MaskCleaning/AsifMaskClean_tellClean/{}_clean_{}.mask'.format(path,star,obs)

    file = '{}00-InputMaterial/NormalizedSpectra/{}_{}.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/normalized_CleanAsif+tellClean/{}_{}.lsd'.format(path,star,obs)

    lsd, mod = pol.run_lsdpy(obs = file, mask = mask, outName = outfile, 
            velStart = velS, velEnd = velE, velPixel = velP, 
            normDepth = 0.2, normLande = 1.2, normWave = 500.0)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean AsifSuperClean

In [10]:
%%capture
with PdfPages('{}03-LSD-calculations/normalized_SuperCleanAsif_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    mask = '{}02-Mask-calculations/MaskCleaning/AsifMaskSuperClean_tellClean/{}_clean_{}.mask'.format(path,star,obs)

    file = '{}00-InputMaterial/NormalizedSpectra/{}_{}.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/normalized_SuperCleanAsif+tellClean/{}_{}.lsd'.format(path,star,obs)

    lsd, mod = pol.run_lsdpy(obs = file, mask = mask, outName = outfile, 
            velStart = velS, velEnd = velE, velPixel = velP, 
            normDepth = 0.2, normLande = 1.2, normWave = 500.0)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)
##~30min

#### **Mask**: tellClean AsifSuperCleanTweak

In [50]:
%%capture
with PdfPages('{}03-LSD-calculations/normalized_SuperCleanAsifTweak_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    mask = '{}02-Mask-calculations/MaskCleaning/AsifMaskSuperCleanTweak_tellClean/{}_clean_{}.mask_auto_twk'.format(path,star,obs)

    file = '{}00-InputMaterial/NormalizedSpectra/{}_{}.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/normalized_SuperCleanAsifTweak+tellClean/{}_{}.lsd'.format(path,star,obs)

    lsd, mod = pol.run_lsdpy(obs = file, mask = mask, outName = outfile, 
            velStart = velS, velEnd = velE, velPixel = velP, 
            normDepth = 0.2, normLande = 1.2, normWave = 500.0)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

### Emission Corrections

In [7]:
df = pd.read_csv('{}03-LSD-calculations/LSD/Params-LSDProfiles-CalculatedByPython.csv'.format(path))

correctedEmission=['hd58050','hd72067','hd120324','hd198183','hd203467']

In [8]:
%%capture
with PdfPages('{}03-LSD-calculations/normalized_SuperCleanAsif_tellClean_2.pdf'.format(path)) as pdf:
  for k in range(len(correctedEmission)):
    for i, ObsInfo in Observations.iterrows():
      star = ObsInfo["Name"]
      if star==correctedEmission[k]:
        obs = ObsInfo["Unnamed: 2"]
        for j in range(0,len(StarData["Name"])):
          if StarData["Name"][j] == star:
            modelcode = StarData["ModelCode"][j].upper()
            if StarData['vsini-estimate'][j]=='very bad':
              vsini = StarData["Adopted-vsini"][j]
            else:
              vsini = StarData['vsini-estimate'][j]
            velS = df["VelStart"][j]
            velE = df["VelEnd"][j]
            velP = df["VelPixel"][j]

        mask = '{}02-Mask-calculations/MaskCleaning/AsifMaskSuperClean_tellClean/{}_clean_{}.mask'.format(path,star,obs)

        file = '{}00-InputMaterial/NormalizedSpectra/{}_{}.s'.format(path,star, obs)
        outfile = '{}03-LSD-calculations/LSD/normalized_SuperCleanAsif+tellClean/{}_{}.lsd'.format(path,star,obs)

        lsd, mod = pol.run_lsdpy(obs = file, mask = mask, outLSDName = outfile, 
                velStart = velS, velEnd = velE, velPixel = velP, 
                normDepth = 0.2, normLande = 1.2, normWave = 500.0)
        
        fig, ax = plt.subplots(4,1, figsize=(10,10))

        ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
        ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
        ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
        ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
        plt.subplots_adjust(hspace=.0)
        ax[3].set_xlabel('Velocity (km/s)')
        ax[3].set_ylabel('I')
        ax[2].set_ylabel('N1')
        ax[1].set_ylabel('N2')
        ax[0].set_ylabel('V')
        fig.suptitle(star + '-'+ str(obs),y=0.92)

        pdf.savefig(fig)
    ##~30min

In [10]:
%%capture
with PdfPages('{}03-LSD-calculations/hybrid_SuperCleanAsif_tellClean2.pdf'.format(path)) as pdf:
  for k in range(len(correctedEmission)):
    for i, ObsInfo in Observations.iterrows():
      star = ObsInfo["Name"]
      if star==correctedEmission[k]:
        obs = ObsInfo["Unnamed: 2"]
        for j in range(0,len(StarData["Name"])):
          if StarData["Name"][j] == star:
            modelcode = StarData["ModelCode"][j].upper()
            if StarData['vsini-estimate'][j]=='very bad':
              vsini = StarData["Adopted-vsini"][j]
            else:
              vsini = StarData['vsini-estimate'][j]
            velS = df["VelStart"][j]
            velE = df["VelEnd"][j]
            velP = df["VelPixel"][j]

        mask = '{}02-Mask-calculations/MaskCleaning/AsifMaskSuperClean_tellClean/{}_clean_{}.mask'.format(path,star,obs)

        file = '{}01-Synth-calculations/Synth-hybrid-spectra/{}_{}_noise.s'.format(path,star, obs)
        outfile = '{}03-LSD-calculations/LSD/hybrid_SuperCleanAsif+tellClean/{}_{}.lsd'.format(path,star,obs)

        lsd, mod = pol.run_lsdpy(obs = file, mask = mask, outLSDName = outfile, 
                velStart = velS, velEnd = velE, velPixel = velP, 
                normDepth = 0.2, normLande = 1.2, normWave = 500.0)
        
        fig, ax = plt.subplots(4,1, figsize=(10,10))

        ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
        ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
        ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
        ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
        plt.subplots_adjust(hspace=.0)
        ax[3].set_xlabel('Velocity (km/s)')
        ax[3].set_ylabel('I')
        ax[2].set_ylabel('N1')
        ax[1].set_ylabel('N2')
        ax[0].set_ylabel('V')
        fig.suptitle(star + '-'+ str(obs),y=0.92)

        pdf.savefig(fig)

    ##~30min